# Konfiguracja

In [1]:
# Algorytmy Grafowe
# Piotr Faliszewski 2019
# Load graph in the DIMACS ascii format + weights

def loadCNFFormula( name ): # Load a CNF formula in the DIMACS ascii format from the file "name" and return it as a list of clauses, 
    V = 0                   # Returns (V,F), V -- highest variable number F -- list of clauses
    L = []  
    f = open( name, "r" )
    lines = f.readlines()
    for l in lines:
        s = l.split()
        if(len(s) < 1): continue
        if( s[0] == "c" ):
            print(s)
            continue
        elif( s[0] == "p" ):
            V = int(s[2])
        else:
            clause = [int(v) for v in s[:-1]]
            L.append(clause)
    f.close()
    return (V,L)

def loadWeightedGraph( name ): # Load a graph in the DIMACS ascii format (with weights) from the file "name" and return it as a list of sets
    V = 0                      # Returns (V,L), V -- number of vertices (1, ..., V), L -- list of edges in the format (x,y,w): edge between x and y with weight w (x<y)"""
    L = []  
    f = open( name, "r" )
    lines = f.readlines()
    for l in lines:
        s = l.split()
        if(len(s) < 1): continue
        if( s[0] == "c" ):
            continue
        elif( s[0] == "p" ):
            V = int(s[2])
        elif( s[0] == "e" ):
                (a,b,c) = (int(s[1]), int(s[2]), int(s[3]))
                (x,y,c) = (min(a,b), max(a,b), c)
                L.append((x,y,c))
    f.close()
    return (V,L)

def loadDirectedWeightedGraph( name ): # Load a directed graph in the DIMACS ascii format (with weights) from the file "name" and return it as a list of sets
    V = 0                              # Returns (V,L), V -- number of vertices (1, ..., V), L -- list of edges in the format (x,y,w): edge between x and y with weight w
    L = []
    f = open( name, "r" )
    lines = f.readlines()
    for l in lines:
        s = l.split()
        if(len(s) < 1): 
            continue
        if( s[0] == "c" ):
            continue
        elif( s[0] == "p" ):
            V = int(s[2])
        elif( s[0] == "e" ):
            (a,b,c) = (int(s[1]), int(s[2]), int(s[3]))
            L.append((a,b,c))
    f.close()
    return (V,L)

def readSolution(name): # Read the expected solution from the first line of the graph file
    with open(name, 'r') as f:
        line = f.readline()
        return line.split()[-1]

In [2]:
import networkx as nx
import matplotlib.pyplot as plt

def show(x): # ((V,L))
    g = nx.Graph()
    temp=[]
    g.add_weighted_edges_from(x[1])
    pos = nx.spectral_layout(g)
    labels = nx.get_edge_attributes(g,'weight')
    nx.draw_networkx_edge_labels(g,pos,edge_labels=labels,font_size=6,rotate=False)
    nx.draw_networkx_nodes(g, pos, node_size=150, node_color='red', alpha=0.6)
    nx.draw_networkx_edges(g, pos, width=0.2, arrowsize=4, alpha=0.9)
    nx.draw_networkx_labels(g, pos, font_size=7, font_family='sans-serif')
    #plt.savefig("graph", dpi=250)

In [3]:
import time
import os

def test(flag):
    file='graphs-lab3/'
    l = list(os.listdir(file))
    cnt=0
    for i in range(len(l)):
        if i==8 and not flag:
            print("\n",l[i],"\n")
            continue
        V, L = loadWeightedGraph(file+l[i])
        G=Graph(V,L)
        time0=time.time()
        res=G.min_cut()
        time1=time.time()
        print("="*50)
        print("| #"+str(i+1),"Wynik:",res,"| Oczekiwane:",readSolution(file+l[i]),"| Czas:",round(time1-time0,3))
        if res==int(readSolution(file+l[i])):
            print("| Test zaliczony!   "+l[i])
            cnt+=1
        else:
            print("| TEST NIEZALICZONY!   "+l[i])
    print("="*50)
    print("Ilosc zaliczonych testów:", str(cnt)+"/"+str(len(l))) 

# Rozwiązanie

### Klasa wierzchołka

In [4]:
class Node:
    def __init__(self):
        self.edges = {}
        self.ver_merged=[]
    
    def add_edge(self, to, weight):
        self.edges[to] = self.edges.get(to,0) + weight
    
    def del_edge(self, to):
        self.edges.pop(to)
    
    def __str__(self):
        return str(list(self.edges.items()))

### Klasa grafu

In [5]:
from queue import PriorityQueue
class Graph:
    def __init__(self,V,L):
        self.V=V
        self.L=L
        self.G=[Node() for i in range(V)]
        for (x,y,c) in L:
            self.G[x-1].add_edge(y-1,c)
            self.G[y-1].add_edge(x-1,c)
            
    def __str__(self):
        res=""
        for v in range(self.V):
            res+= str(v) + (" -> ") + str(self.G[v]) + "  #" + str(self.G[v].ver_merged) + "\n"
        return res

    def show(self):
        show((self.V,self.L))
    
    def merge_vertices(self,x,y):
        self.G[x].ver_merged.append(y)
        for v in self.G[y].edges:
            if x!=v:
                self.G[v].add_edge(x,self.G[y].edges[v])
                self.G[x].add_edge(v,self.G[y].edges[v])
                self.G[v].del_edge(y)
        if self.G[x].edges.get(y) is not None:
            self.G[x].del_edge(y)
        self.G[y].edges.clear()
        self.V-=1
        
    def min_cut_phase(self):
        S=[]
        vis=[False]*(len(self.G))
        for i in range(len(self.G)):
            if len(self.G[i].edges)==0:
                vis[i]=True
        val=[0]*(len(self.G))
        Q=PriorityQueue()
        Q.put((0,0))
        while len(S)!=self.V:
            u=Q.get()
            if not vis[u[1]]:
                for v in self.G[u[1]].edges:
                    if not vis[v]:
                        val[v]+=self.G[u[1]].edges[v]
                        Q.put((-val[v],v))
                vis[u[1]]=True
                S.append(u[1])
        self.merge_vertices(S[-1],S[-2])
        return val[S[-1]]
    
    def min_cut(self):
        res=float("inf")
        while self.V>1:
            res=min(res,self.min_cut_phase())
        return res
        


In [6]:
test(True) 

| #1 Wynik: 99 | Oczekiwane: 99 | Czas: 0.28
| Test zaliczony!   clique100
| #2 Wynik: 19 | Oczekiwane: 19 | Czas: 0.002
| Test zaliczony!   clique20
| #3 Wynik: 199 | Oczekiwane: 199 | Czas: 1.997
| Test zaliczony!   clique200
| #4 Wynik: 4 | Oczekiwane: 4 | Czas: 0.0
| Test zaliczony!   clique5
| #5 Wynik: 2 | Oczekiwane: 2 | Czas: 0.0
| Test zaliczony!   cycle
| #6 Wynik: 1 | Oczekiwane: 1 | Czas: 0.026
| Test zaliczony!   geo100_2a
| #7 Wynik: 2 | Oczekiwane: 2 | Czas: 0.001
| Test zaliczony!   geo20_2b
| #8 Wynik: 1 | Oczekiwane: 1 | Czas: 0.001
| Test zaliczony!   geo20_2c
| #9 Wynik: 2 | Oczekiwane: 2 | Czas: 281.175
| Test zaliczony!   grid100x100
| #10 Wynik: 2 | Oczekiwane: 2 | Czas: 0.002
| Test zaliczony!   grid5x5
| #11 Wynik: 1 | Oczekiwane: 1 | Czas: 0.0
| Test zaliczony!   mc1
| #12 Wynik: 2 | Oczekiwane: 2 | Czas: 0.0
| Test zaliczony!   mc2
| #13 Wynik: 1 | Oczekiwane: 1 | Czas: 0.0
| Test zaliczony!   path
| #14 Wynik: 4 | Oczekiwane: 4 | Czas: 0.057
| Test zaliczony